### データセットの用意

In [1]:
import pandas as pd
from io import StringIO
import sys

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

if (sys.version_info < (3, 0)):
    csv_data = unicode(csv_data)

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [2]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### sklearnによる平均値補完

In [5]:
# from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Imputer

In [7]:
# strategy:meanで平均、medianで中央値、most_frequentで最頻値
imr = Imputer(strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

### 順序特徴量・名義特徴量

In [14]:
# データセットの用意
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


### 順序特徴量のエンコーディング

In [22]:
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


In [23]:
# サイズのマッピング
size_mapping = {'XL': 3,'L': 2,'M': 1}
df1 = df.copy()
df1['size'] = df1['size'].map(size_mapping)
df1

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [26]:
# 逆のマッピング
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df2 = df1.copy()
df2['size'] = df2['size'].map(inv_size_mapping)
df2

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


### 名義特徴量のエンコーディング

In [28]:
# クラスラベルのマッピング
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
print(class_mapping)

# 変換
df4 = df1.copy()
df4['classlabel'] = df['classlabel'].map(class_mapping)
df4

{'class1': 0, 'class2': 1}

In [31]:
# 逆変換
inv_class_mapping = {v: k for k, v in class_mapping.items()}
print(inv_class_mapping)
df5 = df4.copy()
df5['classlabel'] = df5['classlabel'].map(inv_class_mapping)
df5

{0: 'class1', 1: 'class2'}


,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [34]:
# sizeとclasslabelを置換したDF
df4

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


### 名義特徴量のsklearnによる変換

In [36]:
# dfをnparray型に変換
X = df4[['color', 'size', 'price']].values
X

array([['green', 1, 10.1],
       ['red', 2, 13.5],
       ['blue', 3, 15.3]], dtype=object)

In [39]:
# sklaernで変換
from sklearn.preprocessing import LabelEncoder
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

##### ただしこれだとカテゴリ間に順序が生まれてしまうため、あまりよろしくない
one-hotエンコーディングを利用する

In [49]:
# OneHotEncoderによるone-hotエンコーディング
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

In [50]:
# OneHotEncoderによる最初の列の削除
ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()[:, 1:]

array([[ 1. ,  0. ,  1. , 10.1],
       [ 0. ,  1. ,  2. , 13.5],
       [ 0. ,  0. ,  3. , 15.3]])

In [48]:
# get_dummiesによるone-hotエンコーディング
pd.get_dummies(df4[['price','color','size']], drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0
